In [1]:
from aicsimageio import AICSImage
from tifffile import imsave
from pathlib import Path
import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import ball, binary_closing, remove_small_objects, dilation, erosion, disk
from aicssegmentation.core.pre_processing_utils import  intensity_normalization, edge_preserving_smoothing_3d
from aicssegmentation.core.vessel import vesselness3D
from aicssegmentation.core.seg_dot import dot_2d
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
import matplotlib.pyplot as plt
from itkwidgets import view   
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
plt.rcParams["figure.figsize"] = [16, 12]
from aicssegmentation.core.MO_threshold import MO
from aicspylibczi import CziFile

# dots
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects, watershed, dilation, erosion, ball     # function for post-processing (size filter)
from skimage.feature import peak_local_max
from skimage.measure import label
from scipy.ndimage import distance_transform_edt

# lamp1
from aicssegmentation.core.vessel import filament_2d_wrapper
from aicssegmentation.core.seg_dot import dot_2d_slice_by_slice_wrapper
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects, watershed, dilation, erosion, ball  
import warnings
warnings.filterwarnings('ignore')

#shell
from aicssegmentation.core.utils import get_middle_frame, hole_filling, get_3dseed_from_mid_frame
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d

from skimage import filters as skfilters
import numpy as np
import matplotlib.pyplot as plt
from aicsimageio import AICSImage
from skimage import filters as skfilters
from skimage.morphology import binary_erosion, ball, area_opening, opening, cube

In [2]:
# !pip install aicspylibczi>=3.1.1 fsspec>=2022.8.0

In [3]:
# # for tif files 
# filename = r"\\allen\aics\assay-dev\users\Sandi\aics-segmentation\lookup_table_demo\pcna\deconvolution\raw\tiff-decon\session-050823\01PCNA-sc01-p1-e02-check2.ome.tiff"
# reader = AICSImage(filename) 
# IMG = reader.data.astype(np.float32)
# print(IMG.shape)


## for czi files:

# file_path = r"\\allen\aics\assay-dev\users\Sandi\aics-segmentation\lookup_table_demo\pcna\deconvolution\czi-files\session-050823\nearest-neighbor\04PCNA-decon-nn-sc01-p1-e02.czi"
# czi = CziFile(file_path)
# IMG,dims = czi.read_image()
# print(f"shape {IMG.shape} -- dims {dims}")


<h1> Experimental (v11 and v12) <h1>

In [628]:
filename = r"\\allen\aics\assay-dev\users\Sandi\aics-segmentation\lookup_table_demo\pcna\raw-decon\PCNA-decon-ci-S22-P53-E05-20210727.tif"
reader = AICSImage(filename) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


(1, 1, 75, 624, 924)


In [629]:
# # 4 Channel Image
# structure_channel = 1
# struct_img0 = IMG[0,structure_channel,:,:,:].copy()

# 1 Channel Image
structure_channel = 0
struct_img0 = IMG[0,structure_channel,:,:,:].copy()


#view(single_fluorescent_view(struct_img0))

<h1> workflow starts here <h1>

In [630]:
#### PRE-PROCESSING ####

# Intensity scaling and smoothing
intensity_scaling_param = [1, 18] # v11 and 12: [15,18] nn: [0.5,10]
gaussian_smoothing_sigma = 1

# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param) # struct_img0 or pcna_original 

# smoothing with gaussian filter
structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)

In [631]:
#### CORE ALGORITHM ####

# 2D Spot Filter
s2_param = [[0.8, 0.025]] # original: [3, 0.01] #v11: [[1, 0.06]], v12: [[0.8,0.025]]
bw_2dspot = dot_2d_slice_by_slice_wrapper(structure_img_smooth, s2_param)

In [632]:
#### POST-PROCESSING ####

minArea = 20 # v11: 5 v12: 20
seg = remove_small_objects(bw_2dspot>0, min_size=minArea, connectivity=1, in_place=False) #fixed_shape looked good

In [633]:
#### EXTRA PROCESSING #### (v12)

#area opening 
fp1 = ball(1)
fp2 = ball(1)
seg_lvl1 = opening(seg, fp1)
seg_final = opening(seg_lvl1, fp2)


In [634]:
# save file

out=seg_final.astype(np.uint8)
out[out>0]=255

imsave("PCNA-deconseg-S22-P53-E05-20210727-v12.tiff", out) 